In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# importing sklearn libraries
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

# importing tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

### Downloading datasets from kaggle

In [ ]:
with_mask_dir=r'../input/face-mask-dataset/data/with_mask'
without_mask_dir=r'../input/face-mask-dataset/data/without_mask'
filepaths = []
labels= []
dict_list = [with_mask_dir, without_mask_dir]
for i, j in enumerate(dict_list):
    flist=os.listdir(j)
    for f in flist:
        fpath=os.path.join(j,f)
        filepaths.append(fpath)
        if i==0:
          labels.append('with_mask')
        else:
          labels.append('without_mask') 
        
Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
mask_data = pd.concat([Fseries,Lseries], axis=1)
mask_df = pd.DataFrame(mask_data)
print(mask_df.head())
print(mask_df["labels"].value_counts())

In [ ]:
# shape of dataset
mask_df.shape

### Generation of Images for training, validation, and testing using sklearn library module train_test_split.

In [ ]:
train_set, test_images = train_test_split(mask_df, test_size=0.3, random_state=42)
test_set, val_set = train_test_split(test_images, test_size=0.2, random_state=42)

### Generate batches of tensor image data with real-time data augmentation by using ImageDatagenerator from  keras.preprocessing.image library.

In [ ]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)

train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='grayscale',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=32,
                                      shuffle=True            #for the shuffle data
                                     )

test = image_gen.flow_from_dataframe(dataframe= test_set,x_col="filepaths", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='grayscale',
                                     class_mode="categorical",
                                     batch_size=32,
                                     shuffle=True
                                    )

val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'grayscale',
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=True
                                   )

In [ ]:
classes=list(train.class_indices.keys())
for datas in classes:
    print(datas)

#### Function to show images.

In [ ]:
# funciton to show images
def show_images(image_gen):
    test_dict = test.class_indices
    classes = list(test_dict.keys())
    images, labels=next(image_gen) # get a sample batch from the generator 
    plt.figure(figsize=(20,20))
    length = len(labels)
    if length<25:
        r=length
    else:
        r=25
    for i in range(r):
        plt.subplot(5,5,i+1)
        image=(images[i]+1)/2 #scale images between 0 and 1
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color="red",fontsize=16)
        plt.axis('on')
    plt.show()

#### Training, validation, and Testing images.

#### Testing images

In [ ]:
print("Training Images...\n")
show_images(train)

#### Validation Images

In [ ]:
print("Validation Images...\n")
show_images(val)

#### Testing Images

#### The below funciton plot the loss and accuracy of the model.

In [ ]:
print("Testing Images...\n")
show_images(test)

In [ ]:
def plot_loss_and_accuracy(history):
    history_df = pd.DataFrame(history)
    history_df.loc[0:, ['loss', 'val_loss']].plot()
    history_df.loc[0:, ['accuracy', 'val_accuracy']].plot()

## Model Of CNN #1

### Prepare a CNN model and MaxPooling each layers using Sequential Learning model with the help of Keras.

In [ ]:
import keras
from tensorflow.keras import layers

model_CNN_2 = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(244,244,1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

model_CNN_2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_CNN_2.optimizer.lr=0.001

model_CNN_2.summary()
Model: "sequential"

In [ ]:
from keras.callbacks import TensorBoard, ModelCheckpoint
checkpoint = ModelCheckpoint('model_CNN_2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

### Train the model

In [ ]:
history_CNN = model_CNN_2.fit(train, validation_data= val, epochs=20,verbose=1)

### Plotting of loss and accuracy after Training and validation of the CNN model.

In [ ]:
plot_loss_and_accuracy(history_CNN.history)

The above charts show the accuracy and loss for the training set and test set. In the loss chart we see the test set in orange stabilize much faster than the training loss. This same behavior is seen in the accuracy chart. This shows there exists some overfitting in the model. Next, I will adjust the parameters in the second model to overcome the overfitting.

#### Confustion Matrix

In [ ]:
pred = model_CNN_2.predict(test)
pred = np.argmax(pred, axis=1) #pick class with highest  probability

labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2 = [labels[k] for k in pred]

y_test = test_set.labels # set y_test to the expected output
print(classification_report(y_test, pred2))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
plt.figure(figsize = (10,5))
cm = confusion_matrix(y_test, pred2)
sns.heatmap(cm, annot=True, fmt = 'g')

## Saving the model for futher uses.

In [ ]:
model_json = model_CNN_2.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model_CNN_2.save_weights('model.h5')
print('Saved model to disk successfully.')

## Testing the Model

In [ ]:
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
model.load_weights('model.h5')
print('Loaded model from disk successfully')


def classify(img_file):
    img_name = img_file
    test_image = image.load_img(img_name, target_size=(64, 64))

    test_image = image.img_to_array(test_image)  
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)

    if result[0][0] == 1:
        prediction = 'With Mask'
    else:
        prediction = 'Without Mask'
    #print(prediction, img_name)

import os
path = '../../input/test'
files = []

# r=root, d=directories, f=files
#for r, d, f in os.walk(path):
#    for file in f:
#        if '.jpg' in file:
#            files.append(os.path.join(r, file))

#for f in files:
#    classify(f)
#    print('\n')

In [ ]:
classify(test)